In [1]:
from collections import defaultdict 
import pandas as pd
import os
import math

In [2]:
"""PDF Generation Related Imports"""
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4, A3,A2,landscape
from reportlab.lib.units import inch,cm
from reportlab.lib import colors
import itertools

In [3]:
"""Functions Related to Dataframe savin in excel"""
def saveDataFrame(dataframes,filepath,names):
    writer = pd.ExcelWriter(filepath, engine='xlsxwriter')
    for df,sname in zip(dataframes,names):
        df.to_excel(writer,sheet_name=sname)   
    writer.save()

In [ ]:
"""Function Related to PDF Generation"""
def createPage(out_file_name):    
    mycanvas=canvas.Canvas(out_file_name,pagesize=landscape(A4),bottomup =0)
    width, height = landscape(A4)
    width=width-1*inch
    height=height-1*inch
    c_x=1*inch
    c_y=1*inch
    myvalue={"width":width,"height":height,"canvas":mycanvas,'c_x':c_x,'c_y':c_y}
    return myvalue
def writeListVertical(mylist,c_x,c_y,mycanvas,dely):
    #dely=25
    #delx=25
    y=c_y
    x=c_x    
    mycanvas.setFont("Helvetica",8)
    mycanvas.setFillColorRGB(0,0,0.77)
    for txt in mylist:        
        mycanvas.drawString(x,y,str(txt))
        y +=dely
def writeListInVerticalLine(txtlist,ylist,x,mycanvas,cellwidth,cellheight):
    xc=x+math.floor(cellwidth/2)
    for txt,y1 in zip(txtlist,ylist):
        yc=y1+math.floor((1.5*cellheight)/2)
        mycanvas.drawCentredString(xc,yc,str(txt))
def writeListInVerticalLineAllocation(txtlist,ylist,x,mycanvas,cellwidth,cellheight):
    xc=x+math.floor(cellwidth/2)    
    #print(txtlist)
    #print(vallist)
    for txt,y1 in zip(txtlist,ylist):
        if txt>0:
            mystr=str(txt)
        else:
            mystr="" 
        yc=y1+math.floor((1.5*cellheight)/2)
        mycanvas.drawCentredString(xc,yc,mystr)
def writeListInHorizontalLine(txtlist,y,xlist,mycanvas,cellwidth,cellheight,text_align):
    yc=y+math.floor(cellheight/2)
    for txt,x1 in zip(txtlist,xlist):
        xc=x1+math.floor((1.0*cellwidth)/2) 
        mystr=str(txt)
        if text_align==1:            
            mycanvas.drawCentredString(xc,yc,mystr)
        else:
            mycanvas.drawString(xc,yc,mystr)
        
def writeListHorizontal(mylist,c_x,c_y,mycanvas,delx):
    #dely=25
    #delx=25
    y=c_y
    x=c_x
    mycanvas.setFont("Helvetica",fontsize)
    mycanvas.setFillColorRGB(0,0,0.77)    
    for txt in mylist:        
        mycanvas.drawString(x,y,str(txt))
        x +=delx
def drawGridVertical(mylist,xlist,c_y,delta,mycanvas):
    ylist=[c_y+i*delta for i in range(len(mylist)+1)]
    mycanvas.grid(xlist,ylist)
    return ylist

def fillGridVertical(mylist,xlist,c_y,delta,mycanvas,colorlist):
    ylist1=[c_y+i*delta for i in range(len(mylist)+1)]
    ylist=ylist1[1:]
    color_index=[i%7 for i in range(0,len(ylist))]
    mycolors=[colorlist[i] for i in color_index]
    x=xlist[0] 
    vallist=mylist[1:]
    for val,y,fill_color in zip(vallist,ylist,mycolors):
        if val >0:
            mycanvas.setFillColor(fill_color)
            mycanvas.rect(x,y,delta,delta,fill=1)
        
    
def writeHeading(dframe,canvas_name):
    code=list(dframe['Code'])
    code.insert(0,"code")
    names=list(dframe['Resource_name'])
    names.insert(0,'Resource_name')
    units=list(dframe['Unit'])
    units.insert(0,"Unit")
    c_x=1.0*inch
    c_y=1.0*inch
    xlist=[1*inch,1.5*inch]
    delta=25
    fontsize=10
    """Setting fontsie and color"""
    canvas_name.setFont("Helvetica",fontsize)
    canvas_name.setFillColorRGB(0,0,0.77)    
    ylist=drawGridVertical(code,xlist,c_y,delta,canvas_name)
    #writeListVertical(code,c_x,c_y,canvas_name,25)
    writeListInVerticalLine(code,ylist,c_x,canvas_name,0.5*inch,25)
    c_x=1.5*inch
    xlist=[1.5*inch,3.0*inch]
    #writeListVertical(names,c_x,c_y,canvas_name,25)
    ylist=drawGridVertical(code,xlist,c_y,delta,canvas_name)
    writeListInVerticalLine(names,ylist,c_x,canvas_name,1.5*inch,25)
    c_x=3.0*inch
    xlist=[3.0*inch,3.7*inch]
    ylist=drawGridVertical(units,xlist,c_y,delta,canvas_name)
    writeListInVerticalLine(units,ylist,c_x,canvas_name,0.7*inch,25)
    #c_x=3.0*inch
    #writeListVertical(units,c_x,c_y,canvas_name,25)
    return 3.7*inch

def writeDayHeading(start_day,finish_day,cellWidth,cellHeight,canvas_name,c_x,pno):
    daytext=dayTextGenerator(start_day,finish_day)
    print(daytext)
    if pno>1:
        start_day=start_day-(pno-1)*20
        finish_day=finish_day-(pno-1)*20
    multiplier=[i%21 for i in range(start_day,finish_day+1)]
    xlist=[c_x+(i-1)*cellWidth for i in multiplier]
    c_y=1.0*inch
    print("xlist={}".format(xlist))
    print("days={}".format(daytext))
    fontsize=8
    canvas_name.setFont("Helvetica",fontsize)
    #canvas_name.setFillColorRGB(1,1,1)
    writeListInHorizontalLine(daytext,c_y,xlist,canvas_name,cellWidth,cellHeight,1)
    
def writeDailyAllocationData(dframe,canvas_name,c_x,start_day,end_day,cellWidth,cellHeight):
    daytext=dayTextGenerator(start_day,end_day)
    x1=c_x
    c_y=1.0*inch
    delta=cellHeight
    fontsize=6
    """Setting fontsie and color"""
    canvas_name.setFont("Helvetica",fontsize)
    canvas_name.setFillColorRGB(0,0,0.77) 
    myclolor_list=createColorList()
    print(myclolor_list)
    for day in daytext:        
        vlist=list(dframe[day])
        vlist.insert(0,"")
        x2=x1+cellWidth
        xlist=[x1,x2]
        ylist=drawGridVertical(vlist,xlist,c_y,delta,canvas_name)
        fillGridVertical(vlist,xlist,c_y,delta,canvas_name,myclolor_list)
        canvas_name.setFillColorRGB(0,0,0)
        vlist1=vlist[1:]
        ylist1=ylist[1:]
        writeListInVerticalLineAllocation(vlist1,ylist1,x1,canvas_name,cellWidth,cellHeight)
        x1+=cellWidth
        print(vlist)

In [4]:
class GanttBar:
    def __init__(self,start,duration,name="",unit="",volume=0):
        self.taskName=name
        self.start_day=start
        self.duration=duration
        self.finish_day=self.start_day+self.duration
        self.taskUnit=unit
        self.taskVolume=volume
        self.productionRate=math.ceil(self.taskVolume/self.duration)
    def addTaskUnit(self,unit):
        self.taskUnit=unit
    def addTaskVolume(self,volume):
        self.taskVolume=volume
        self.productionRate=math.ceil(self.taskVolume/self.duration)
        

In [5]:
"""Graph Driver"""
def text2List(input_text):
    split_text=input_text.split(',')
    output_list=[int(x) for x in split_text ]
    return output_list
    
def buildGraph(input_df):
    shape=input_df.shape
    g=Graph(shape[0])
    print(shape)
    for index,row in input_df.iterrows():
        v=row['TaskNo']
        if row['Predecessor']!=-1:
            pred_text=str(row['Predecessor'])
            length=len(pred_text)
            if length >1:
                sources=text2List(pred_text)
                #print(sources)
            else:
                sources=[int(pred_text)]
            #print("sources={}".format(sources))
            for u in sources:
                g.addEdge(u,v)
            #print("predecosseo no={} value={}".format(length,pred_text))
            #print(len(pred_text))
            #print(pred_text)
    duration_list=list(input_df['Duration'])
    g.addDurations(duration_list)

    return g


In [6]:
"""Class to represent graph"""
class Graph:
    def __init__(self,vertices):
        self.graph=defaultdict(list)
        self.V=vertices
        self.indeg0=[True]*(self.V+1)
        self.outdeg0=[True]*(self.V+1)
        self.edges=[]
        self.adj=defaultdict(list)
        self.visited=[False]*(self.V+1)
        self.path=[]
        self.durations=[]
        self.taskchains=[]
        self.tc_durations=[]
        self.minimum_start_date=[0]*self.V
    # function to add an edge to graph 
    def addEdge(self,u,v):
        self.graph[u].append(v)
        edge=(u,v)
        print(edge)
        self.edges.append(edge)
        # set indeg0[v] <- false 
        self.indeg0[v] = False  
        # set outdeg0[u] <- false 
        self.outdeg0[u] = False
        #print(self.graph[u])
    # A recursive function used by topologicalSort 
    def topologicalSortUtil(self,v,visited,stack):
        #mark the current node as visited
        visited[v]=True
        # Recur for all the vertices adjacent to this vertex      

        for i in self.graph[v]:
            if visited[i]==False:
                self.topologicalSortUtil(i,visited,stack)
         # Mark the current node as visited. 
        stack.insert(0,v)
    # The function to do Topological Sort. It uses recursive  
    # topologicalSortUtil() 
    def topologicalSort(self):
        # Mark all the vertices as not visited
        visited=[False]*(self.V+1)
        stack=[]
        # Call the recursive helper function to store Topological 
        # Sort starting from all vertices one by one 
        for i in range(self.V):
            if visited[i]==False:
                self.topologicalSortUtil(i,visited,stack)
        print (stack)
    def displayGraph(self):
        print("number of vertices={}".format(self.V))
        print(self.graph)
        print("indegree={}".format(self.indeg0))
        print("outdegree={}".format(self.outdeg0))
        print("edges of the graph......")
        print(self.edges)
        print("durations of tasks")
        print(self.durations)
    def dfs(self,s):
        k=0       
        #append the node in path
        #and set visited
        self.path.append(s)
        self.visited[s]=True
        # Path started with a node 
        # having in-degree 0 and 
        # current node has out-degree 0, 
        # print current path 
        if self.outdeg0[s] and self.indeg0[self.path[0]]:
            print(*self.path)
            self.taskchains.append(list(self.path))
            #for p in self.path:
                #self.taskchains[k].append(p)               
            #myvalue[k]=self.path
            #
        k=k+1
            #print(myvalue)
        # Recursive call to print all paths
        for node in self.graph[s]:
            if not self.visited[node]:
                self.dfs(node)
        # Remove node from path 
        # and set unvisited
        #return self.path
        #print(self.path)
        #self.taskchains[s]=self.path
        self.path.pop()
        self.visited[s]=False
        #return myvalue
    def print_all_paths(self):
        for i in range(self.V):
            if self.indeg0[i] and self.graph[i]:
                self.path=[]
                self.visited=[False]*(self.V+1)
                self.dfs(i)
                #print("path={}".format(p))
        
    def addDurations(self,duration_list):
        self.durations=duration_list
        
    def findPathdurations(self,path):
        total_duration=0
        for p in path:
            total_duration +=self.durations[p-1]
        return total_duration
    def findPathCumulativeDurations(self,path):
        total_duration=0
        path_cum=[]
        for p in path:
            total_duration +=self.durations[p-1]
            path_cum.append(total_duration) 
        self.tc_durations.append(path_cum)
        #return path_cum
    def findStratDateofTasks(self):
        for T in range(1,self.V+1):
            d=self.findMinimumStartDateOfTask(T)
            print("Task-{} sdate={}".format(T,d))
            self.minimum_start_date[T-1]=d
    def findMinimumStartDateOfTask(self,T):
        d=self.durations[T-1]
        duration_list=[]
        for tc,tcd in zip(self.taskchains,self.tc_durations):
            if T in tc:
                index=tc.index(T)
                sdate=tcd[index]-d+1
                duration_list.append(sdate)
                #print("task found in={}".format(index))
            #print("chain={} duration={}".format(tc,tcd))
        #print("duration of tasks no={} total_time={}".format(T,d))
        max_start_date=max(duration_list)
        return max_start_date
    def calculateStartOfAllTasks(self):
        for chain in self.taskchains:
            #duration=self.findPathdurations(chain)
            self.findPathCumulativeDurations(chain)
           
    def scheduleTask(self):
        self.topologicalSort() 
        #self.displayGraph()
        self.print_all_paths()
        print(self.taskchains)
        for chain in self.taskchains:
            #duration=self.findPathdurations(chain)
            self.findPathCumulativeDurations(chain)
        #self.calculateStartOfAllTasks()
        #print(self.tc_durations) 
        self.findStratDateofTasks() 
        self.findProjectDuration()
    def createGanttSchedule(self,names,units,volumes):
        mytasks=[]
        for i in range(1,self.V+1):            
            duration=self.durations[i-1]
            sday=self.minimum_start_date[i-1]
            unit=units[i-1]
            volume=volumes[i-1]
            name=names[i-1]
            bar=GanttBar(sday,duration,name,unit,volume)
            mytasks.append(bar)
        return mytasks
    def findProjectDuration(self):
        project_durations=[]
        for tcd in self.tc_durations:
            dmax=max(tcd)
            #print(dmax)
            project_durations.append(dmax)
        self.finishDay=max(project_durations)
            
            
            
        
   
   
        
        


In [7]:
#myfolder=r'F:\website\cmis6\Civilworks cost\Work Schedule' #office computer
myfolder=r'E:\Website_26_07_2020\cmis6\Civilworks cost\Work Schedule' #home computer
inputpath=os.path.join(myfolder,'Schdule Input.xlsx')
outputpath_for_dframe=os.path.join(myfolder,'dframes.xlsx')

myframes=[]
mynames=[]



sheetName="Main"
main_df=pd.read_excel(inputpath,sheet_name=sheetName)
main_df
for index,row in main_df.iterrows():
    sheetName=row['Allocation_Sheet_Name']
    task_df=pd.read_excel(inputpath,sheet_name=sheetName)
    task_df.fillna(0,inplace=True)
    g=buildGraph(task_df)
    g. scheduleTask()
    print(g.finishDay)
    tnames=list(task_df['Name'])
    units=list(task_df['Unit'])
    volumes=list(task_df['Volume'])
    print("unit={} volumes={}".format(tnames,units,volumes))
    mybars=g.createGanttSchedule(tnames,units,volumes)
    g.findProjectDuration()
    #print(task_df)
    print("before adding unit......")
      
    for bar in mybars:
        print("name={} unit={} volume={} production={}".format(bar.taskName,bar.taskUnit,
                                                               bar.taskVolume,bar.productionRate))
    myframes.append(task_df)
    mynames.append(sheetName)
saveDataFrame(myframes,outputpath_for_dframe,mynames)
        
    




(10, 6)
(1, 2)
(1, 3)
(2, 4)
(3, 4)
(1, 5)
(5, 6)
(6, 7)
(7, 8)
(3, 9)
(8, 9)
(9, 10)
[1, 5, 6, 7, 8, 3, 9, 10, 2, 4, 0]
1 2 4
1 3 4
1 3 9 10
1 5 6 7 8 9 10
[[1, 2, 4], [1, 3, 4], [1, 3, 9, 10], [1, 5, 6, 7, 8, 9, 10]]
Task-1 sdate=1
Task-2 sdate=8
Task-3 sdate=8
Task-4 sdate=24
Task-5 sdate=8
Task-6 sdate=42
Task-7 sdate=52
Task-8 sdate=62
Task-9 sdate=72
Task-10 sdate=82
88
unit=['Mobilization and Site Preparation', 'CC Block Manufacture(30X30X30)', 'CC Block Manufacture(40X40X20)', 'Curing CC Block', 'Embankment Construction', 'Placing Sand Filter', 'Placing Geotextile Filter', 'Placing Pea Gravel Filter', 'Placing CC Block', 'Ancilary Work for Site Completion'] volumes=['Percent', 'Nos', 'Nos', 'Percent', 'Cum', 'Cum', 'sqm', 'Cum', 'Nos', 'Percent']
before adding unit......
name=Mobilization and Site Preparation unit=Percent volume=100 production=15
name=CC Block Manufacture(30X30X30) unit=Nos volume=33338 production=2084
name=CC Block Manufacture(40X40X20) unit=Nos volume=5012 pr

In [8]:
#sheetName='WBS'
#myframe=pd.read_excel(inputpath,sheet_name=sheetName)
#myframe.fillna(0,inplace=True)
#myframe

In [9]:
#g=buildGraph(myframe)
#g. scheduleTask()
#print(g.finishDay)
#mybars=g.createGanttSchedule()
#g.findProjectDuration()

In [10]:
#g.findProjectDuration()